In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [2]:
def conv_block(tensor_in, filters, kernel_size, repeats=2, pool_strides=(2, 2), block_id=1):
    x = tensor_in
    for i in range(repeats):
        conv_name = f'block{block_id}_conv{i + 1}'
        x = Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same', name=conv_name)(x)

    x = MaxPooling2D((2, 2), strides=pool_strides, name=f'block{block_id}_pool')(x)
    return x

In [3]:
def create_vggnet(input_shape=(224, 224, 3), n_classes=2):
    input_tensor = Input(shape=input_shape, name='input')
    x = conv_block(input_tensor, filters=64, kernel_size=(3, 3), repeats=2, pool_strides=(2, 2), block_id=1)
    x = conv_block(x, filters=128, kernel_size=(3, 3), repeats=2, pool_strides=(2, 2), block_id=2)
    x = conv_block(x, filters=256, kernel_size=(3, 3), repeats=3, pool_strides=(2, 2), block_id=3)
    x = conv_block(x, filters=512, kernel_size=(3, 3), repeats=3, pool_strides=(2, 2), block_id=4)
    x = conv_block(x, filters=512, kernel_size=(3, 3), repeats=3, pool_strides=(2, 2), block_id=5)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5, name='dropout_1')(x)
    x = Dense(units=120, activation='relu', name='fc_1')(x)
    x = Dropout(0.5, name='dropout_2')(x)

    output = Dense(units=n_classes, activation='sigmoid', name='output')(x)
    model = Model(inputs=input_tensor, outputs=output, name='vggnet')
    model.summary()

    return model

In [4]:
from cnn.utils.image_handler import ImageHandler

img_srcs = [
    '/Users/jiheo/Desktop/workspace/datasets/cpfd/images/clean_door/open',
    '/Users/jiheo/Desktop/workspace/datasets/cpfd/images/clean_door/closed',
]

classes = [0, 1]

img_handler = ImageHandler(src_dirs=img_srcs, classes=classes, batch_size=128)

[INFO] reading src `/Users/jiheo/Desktop/workspace/datasets/cpfd/images/clean_door/open`
[INFO] reading src `/Users/jiheo/Desktop/workspace/datasets/cpfd/images/clean_door/closed`
[INFO] finished reading images: len of images 2000, len of labels 2000
image shape: (2000, 224, 224, 3)
label shape: (2000,)
[INFO] scaling images
[INFO] train_images, train_labels, test_images, test_labels splitted
[INFO] each shape: (1500, 224, 224, 3), (1500, 2), (500, 224, 224, 3), (500, 2)


Model: "vggnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0    

2022-04-07 11:45:32.063107: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-07 11:45:32.063342: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
model = create_vggnet(input_shape=(224, 224, 3), n_classes=2)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, mode='min', verbose=1)
ely_cb = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)

history = model.fit(
    img_handler.train_ds,
    epochs=20,
    validation_data=img_handler.valid_ds,
    callbacks=[rlr_cb, ely_cb]
)

2022-04-07 11:25:44.728773: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/20
10/10 [==============================] - 435s 43s/step - loss: 0.6931 - accuracy: 0.5136 - val_loss: 0.6954 - val_accuracy: 0.4600
Epoch 2/20
